In [ ]:
!pip install tensorflow

In [ ]:
!pip install sklearn

# CIFAR-10 Computer Vision Model Training and Evaluation

This notebook implements a Convolutional Neural Network (CNN) for image classification using the CIFAR-10 dataset. The code demonstrates a complete machine learning pipeline from data loading to model evaluation.

## Key Components:

### 1. **Library Imports**
- **TensorFlow/Keras**: Deep learning framework for building and training the CNN
- **NumPy**: Numerical operations and array handling
- **Scikit-learn**: Evaluation metrics (accuracy, precision, recall, F1-score, confusion matrix)
- **OS**: File system operations for saving the trained model
- **Logging**: Training progress and debugging information

### 2. **Data Preprocessing**
- Loads CIFAR-10 dataset (60,000 32x32 color images in 10 classes)
- Normalizes pixel values from [0,255] to [0,1] range for better training performance
- Uses sparse categorical labels (integers 0-9) instead of one-hot encoding

### 3. **CNN Architecture**
- **Input Layer**: 32x32x3 (RGB images)
- **Convolutional Layers**: Two Conv2D layers (32 and 64 filters) with ReLU activation
- **Pooling Layers**: MaxPooling2D for dimensionality reduction
- **Dense Layers**: Fully connected layers ending with 10-class softmax output

### 4. **Optimizer Comparison**
The code includes three optimizer options with performance characteristics:
- **SGD**: Slower convergence, needs 50+ epochs, achieves ~65-70% accuracy
- **RMSprop**: Medium convergence, ~30 epochs, achieves ~60-65% accuracy  
- **Adam**: Fast convergence, ~20-30 epochs, achieves ~60-65% accuracy

### 5. **Training Process**
- Compiles model with sparse categorical crossentropy loss
- Trains with validation on test set to monitor overfitting
- Saves trained model to `/saved_model/cifar10_model.keras`

### 6. **Model Evaluation**
Comprehensive evaluation using multiple metrics:
- **Accuracy**: Overall classification correctness
- **Precision**: True positives / (True positives + False positives)
- **Recall**: True positives / (True positives + False negatives)
- **F1-Score**: Harmonic mean of precision and recall
- **Confusion Matrix**: Detailed breakdown of classification results

### 7. **Key Technical Details**
- Uses `np.argmax()` to convert probability predictions to class labels
- Handles label format conversion (flattening for sparse labels)
- Implements weighted averaging for multi-class metrics
- Includes TensorBoard logging for hyperparameter visualization

This implementation provides a solid foundation for image classification tasks and demonstrates best practices for CNN training and evaluation.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os  # Add this import since you're using os.makedirs
import numpy as np
import sklearn.metrics
#from tensorflow.keras.layers import Input
#from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
import logging

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

logging.getLogger().setLevel(logging.INFO)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

METRIC_ACCURACY = "accuracy"
validation = 'validation'
logging.getLogger().setLevel(logging.INFO)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)


def train_cifar_model(epochs, batch_size, optimizer):
    """
    This function contains the logic from your keras_cifar10.py script.
    """
    # 1. Load data
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

    # Normalize pixel values
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0

    # 2. Define the Keras model - Use Input layer instead of input_shape
    model = keras.Sequential([
        keras.layers.Input(shape=(32, 32, 3)),  # Add Input layer first
        keras.layers.Conv2D(32, (3, 3), activation='relu'),  # Remove input_shape parameter
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])

    # Typical results for CIFAR-10:
    ##SGD:     Slower convergence, may need 50+ epochs, final accuracy ~65-70%
    ##RMSprop: Medium convergence, ~30 epochs, final accuracy ~60-65%  
    ##Adam:    Fast convergence, ~20-30 epochs, final accuracy ~60-65%
    ##Learning Rate Sensitivity:
    ##SGD:     Very sensitive - wrong LR can break training
    ##RMSprop: Moderately sensitive - usually works with default
    ##Adam:    Least sensitive - 0.001 works for most problems


    learning_rate = 0.001

    
    loss_param="categorical_crossentropy"
    loss_param='sparse_categorical_crossentropy'

    #optimizer = "sgd"
    #optimizer = "rmsprop"
    optimizer = "adam"
    #opt = None
    if optimizer == "sgd":
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == "rmsprop":
        opt = RMSprop(learning_rate=learning_rate)
    elif optimizer == "adam":
        opt = Adam(learning_rate=learning_rate)
    else:
        raise Exception("Unknown optimizer", optimizer)

    # 3. Compile the model
    model.compile(#optimizer=optimizer,
                  loss=loss_param,
                  metrics=['accuracy'],
                  optimizer=opt
    )

    # 4. Train the model
    history = model.fit(x_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(x_test, y_test))
    print(f"Model History: {history}")

    # 5. Save the trained model
    os.makedirs('/saved_model', exist_ok=True)
    model_save_path = '/saved_model/cifar10_model.keras' 
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    # -- model.summary()
    
    # Convert the lists of uniform elements into NumPy arrays
    test_x = x_test # np.array(test_x_list)
    test_y = y_test # np.array(test_y_list)

    # Use the model to predict the labels
    test_predictions = model.predict(test_x)
    test_y_pred = np.argmax(test_predictions, axis=1)
    ###test_y_true = np.argmax(test_y, axis=1) - incorrect
    test_y_true = test_y.flatten()  # Correct
    # or
    ##test_y_true = test_y.squeeze()  # Also correct


    y_pred = np.argmax(test_predictions, axis=1)  # Convert probabilities to class predictions
    y_true = np.argmax(y_test, axis=1)  # Convert one-hot encoded labels to class indices

    # Evaluating model accuracy and logging it as a scalar for TensorBoard hyperparameter visualization.
    accuracy = sklearn.metrics.accuracy_score(test_y_true, test_y_pred)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
    #logging.info("Test accuracy:{}".format(accuracy))

    print("Test accuracy:{}".format(accuracy))

    ## Diff test accuracy

    # Evaluate
    ## 2

    #result = test_predictions #model.predict(test_y_true, test_y_pred)

    #print(result)
    ''' ''' 
    y_true = test_y_true
    y_pred = test_y_pred
    
    # Calculate metrics
    metrics = {
    "accuracy": accuracy_score(y_true, y_pred),
    
    "precision": precision_score(y_true, y_pred, average='weighted'),  # or 'macro', 'micro'
    
    "recall": recall_score(y_true, y_pred, average='weighted'),
    
    "f1_score": f1_score(y_true, y_pred, average='weighted'),
    
    "confusion_matrix": confusion_matrix(y_true, y_pred).tolist()
    }


    return f"The Score for the computer vision model:\n {metrics}" # history


In [ ]:
#Run to train the model

# Typical results for CIFAR-10:
##SGD:     Slower convergence, may need 50+ epochs, final accuracy ~65-70%
##RMSprop: Medium convergence, ~30 epochs, final accuracy ~60-65%  
##Adam:    Fast convergence, ~20-30 epochs, final accuracy ~60-65%
##Learning Rate Sensitivity:
##SGD:     Very sensitive - wrong LR can break training
##RMSprop: Moderately sensitive - usually works with default
##Adam:    Least sensitive - 0.001 works for most problems

hyperparameters = {"epochs": 30, "batch-size": 256, "optimizer": 'adam'}

train_cifar_model('', hyperparameters["epochs"], hyperparameters["batch-size"], hyperparameters["optimizer"])

## Image Preprocessing - Essential Requirements

This preprocessing step is **essential** because:

- **Standardization**: All images must be the same size for batch processing
- **Model Compatibility**: The CNN was trained on 32×32 images, so new images must match  
- **Memory Efficiency**: Smaller images reduce computational requirements
- **Consistency**: Ensures the model receives data in the expected format

> **Note**: This resizing step is critical before feeding images to our trained CIFAR-10 model for accurate predictions.

In [ ]:
from PIL import Image
image_path = '0009.jpg' 
img = Image.open(image_path).resize((32, 32))

img

## CIFAR-10 Image Prediction System

- This notebook demonstrates how to load a trained CIFAR-10 model and make predictions on new images using Snowflake's Python environment.
- Load a saved Keras model and predict which of the 10 CIFAR-10 classes a new image belongs to, with complete preprocessing pipeline.
- Predict single image
- Batch Processing predict images

In [ ]:
import tensorflow as tf
import numpy as np
# You might need to install Pillow: pip install Pillow
from PIL import Image

# Define the human-readable class names for CIFAR-10
CLASS_NAMES = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']
local_model_path = '/tmp/keras_cifar10_model'

def predict_single_image(model_path, image_path):
    """
    Loads a saved model, preprocesses a single image,
    and returns the predicted class name.
    
    Args:
        model_path (str): The path to the saved Keras model directory.
        image_path (str): The path to the new image file.
        
    Returns:
        str: The predicted class name.
    """
    # --- 1. Load the saved model ---
    # Use tf.keras.models.load_model to load the entire model.
    print(f"Loading model from: {model_path}")
    model = tf.keras.models.load_model(model_path)
    
    # --- 2. Load and Preprocess the Image ---
    # The model was trained on 32x32 images, so we must resize our new image.
    img = Image.open(image_path).resize((32, 32))
    
    # Convert the image to a NumPy array and normalize pixel values to [0, 1]
    img_array = np.array(img) / 255.0
    
    # The model.predict method expects a "batch" of images.
    # We add a new axis to turn our (32, 32, 3) image into (1, 32, 32, 3).
    img_batch = np.expand_dims(img_array, axis=0)
    
    # --- 3. Make the Prediction ---
    predictions = model.predict(img_batch)
    
    # --- 4. Interpret the Results ---
    # The output is an array of probabilities for each class.
    # We find the index of the highest probability using np.argmax.
    predicted_class_index = np.argmax(predictions[0])
    
    # Map the index to its corresponding class name.
    predicted_class_name = CLASS_NAMES[predicted_class_index]
    
    return predicted_class_name

# --- Example Usage ---
if __name__ == '__main__':
    # First, run your training process
    ##main()
    
    # Now, use the function to predict on a new image
    # IMPORTANT: Replace these paths with your actual paths
    SAVED_MODEL_PATH = '/saved_model/cifar10_model.keras' # '/tmp/keras_cifar10_model'
    # Download or find a sample image of a truck, car, etc.
    NEW_IMAGE_PATH = '0009.jpg' #plane
    #NEW_IMAGE_PATH = '0016.jpg' # cat
    #NEW_IMAGE_PATH = '0007.jpg' # ship

    # Create a list of all images you want to test
    images_to_test = ['0007.jpg', '0009.jpg', '0016.jpg'] # ship, plane, cat
    
    for image_file in images_to_test:
        try:
            print(f"\n--- Predicting for image: {image_file} ---")
            prediction = predict_single_image(SAVED_MODEL_PATH, image_file)
            print(f"📸 The model predicts this image is a: {prediction.upper()}")
        except FileNotFoundError:
            print(f"Error: Image file not found at '{image_file}'.")

    try:
        prediction = predict_single_image(SAVED_MODEL_PATH, NEW_IMAGE_PATH)
        print("\n" + "="*30)
        print(f"📸 The model predicts the image is a: {prediction.upper()}")
        print("="*30)
    except FileNotFoundError:
        print(f"\nError: Image file not found at '{NEW_IMAGE_PATH}'.")
        print("Please update NEW_IMAGE_PATH with a valid path to an image.")